In [9]:
!pip install import_ipynb
import import_ipynb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.6 MB 5.0 MB/s 


In [10]:
import tensorflow
from tensorflow.keras.layers import Conv2D,BatchNormalization,Activation,Add,ZeroPadding2D,\
MaxPooling2D,AveragePooling2D,Input,Flatten,Dense
from tensorflow.keras.initializers import GlorotUniform
from keras.models import Model
from tensorflow.keras.preprocessing import image

import numpy as np
import pandas as pd
import os
from tqdm import tqdm
from utils import save_model_arc

In [2]:
def conv_norm(X,filter_,kernel_size_=(1,1),stride_=(1,1),padding_="valid",axis_=3):
    X = Conv2D(filters=filter_, kernel_size=kernel_size_, strides=stride_, padding=padding_)(X)
    X = BatchNormalization(axis=axis_)(X)
    return X

In [3]:
def identity_block(X,f,filters):
    F1,F2,F3 = filters
    X_shortcut = X
    
    # layer 1
    X = conv_norm(X,filter_=F1)
    X = Activation("relu")(X)
    
    # Layer 2
    X = conv_norm(X,filter_=F2,kernel_size_=(f,f),padding_="same")
    X = Activation("relu")(X)
    
    # Layer 3
    X = conv_norm(X,filter_=F3)
    
    # Skip connection
    X = Add()([X,X_shortcut])
    X = Activation("relu")(X)
    
    return X

In [4]:
def identity_blocks(X,f,filters,repeat=1):
    for _ in range(repeat):
        X = identity_block(X,f,filters)
    return X

In [5]:
def convolution_block(X,f,filters,s=2):
    F1,F2,F3 = filters
    X_shortcut = X
    
    # layer 1
    X = conv_norm(X,filter_=F1,stride_=(s,s))
    X = Activation("relu")(X)
    
    # Layer 2
    X = conv_norm(X,filter_=F2,kernel_size_=(f,f),padding_="same")
    X = Activation("relu")(X)
    
    # Layer 3
    X = conv_norm(X,filter_=F3)
    
    # Skip connection
    X_shortcut = conv_norm(X_shortcut,filter_=F3,stride_=(s,s))
    X = Add()([X,X_shortcut])
    X = Activation("relu")(X)
    
    return X

In [6]:
def ResNet_(classes,input_shape=(224,224,3)):
    # Conv2D->BATCH_NORM->RELU->MAXPOOL->CONV_BLOCK->ID_BLOCK*2->CONV_BLOCK->ID_BLOCK*3->
    # CONV_BLOCK->ID_BLOCK*5->CONV_BLOCK->ID_LOCK*2->AVG_POOL->FC_LAYER
    
    X_input = Input(input_shape)
    
    # zero padding
    X = ZeroPadding2D(padding=(3,3))(X_input)
    
    # stage 1
    X = conv_norm(X,filter_=64,kernel_size_=(7,7),stride_=(2,2))
    X = Activation("relu")(X)
    X = MaxPooling2D(pool_size=(3,3),strides=(2,2))(X)
    
    # stage 2
    X = convolution_block(X,f=3,filters=[64,64,256],s=1)
    X = identity_blocks(X,f=3,filters=[64,64,256],repeat=2)
    
    # stage 3
    X = convolution_block(X,f=3,filters=[128,128,512],s=2)
    X = identity_blocks(X,f=3,filters=[128,128,512],repeat=3)
    
    # stage 4
    X = convolution_block(X,f=3,filters=[256,256,1024],s=2)
    X = identity_blocks(X,f=3,filters=[256,256,1024],repeat=5)
    
    # stage 5
    X = convolution_block(X,f=3,filters=[512,512,2048],s=2)
    X = identity_blocks(X,f=3,filters=[512,512,2048],repeat=2)
    
    # Avg-Pooling
    X = AveragePooling2D(pool_size=(2,2),name="avg_pool")(X)
    
    # Output Layer
    X = Flatten()(X)
    X = Dense(classes,activation="softmax",name="fc"+str(classes))(X)
    
    # create model
    model = Model(inputs=X_input, outputs=X,name="ResNet_")
    
    return model

In [7]:
model = ResNet_(input_shape=(224,224,3),classes=57)

In [8]:
model.summary()

Model: "ResNet_"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 zero_padding2d (ZeroPadding2D)  (None, 230, 230, 3)  0          ['input_1[0][0]']                
                                                                                                  
 conv2d (Conv2D)                (None, 112, 112, 64  9472        ['zero_padding2d[0][0]']         
                                )                                                                 
                                                                                            

In [11]:
save_model_arc(model)